 # VARIABLES

In [ ]:
background_database_name = "forwast"

product_system_name = 'AP_LLM'
product_system_database_name = f'{product_system_name}_{background_database_name}'

xls_file_path = 'LCI_AP_LLM.xlsx'

excluded_sheets = ["SheetNames"]

project_name = "forwast"
FG_DB = product_system_database_name
BG_DB = background_database_name

LLM_mapping = True

# IMPORTS

In [ ]:
import sys
import os
import pandas as pd
import brightway2 as bw

from bw2io.strategies import (
    normalize_units,    
    remove_zero_amount_coproducts,
    normalize_biosphere_categories,
    normalize_biosphere_names,
    strip_biosphere_exc_locations,)

sys.path.insert(0, os.path.abspath(".."))
import fauldier as fd

# RESTORE PROJECT

In [ ]:
if project_name not in bw.projects:
    bw.restore_project_directory(fp="forwast.gz", project_name=project_name)

# SPREADSHEET TO BRIGHTWAY

In [ ]:
if LLM_mapping:
    bw.projects.set_current(project_name)
    db = bw.Database('biosphere3')

    # Filter for keywords
    #keywords = []

    # Create a list of activity names
    biosphere_flows = [
        f"{act['name']} | {act['categories']} | {act['unit']}"
        for act in db
        #if any(keyword in act['name'].lower() for keyword in keywords)
    ]  

In [ ]:
biosphere_flows 

In [ ]:
if LLM_mapping:
    db = bw.Database(background_database_name)

    # Filter for keywords
    #keywords = []
    #allowed_locations = ['GLO']
    
    # Create a list of activity names
    BG_DB_activities = [
        f"{act['name']} | {act.get('reference product', act['name'])} | {act['location']} | {act['unit']}"
        for act in db
        #if any(keyword in act['reference product'].lower() for keyword in keywords)
        #if act.get('location') in allowed_locations
    ]

else:
    BG_DB_activities = []

In [ ]:
 BG_DB_activities

In [ ]:
%%time
fd.x2bw_transformation(product_system_database_name, xls_file_path, excluded_sheets, background_database_name, LLM_mapping, BG_DB_activities, biosphere_flows)

# WRITE DATABASE

## Imports and Project

In [ ]:
bw.projects

In [ ]:
bw.projects.set_current(project_name)

In [ ]:
list(bw.databases)

## Import sheet and apply strategies

In [ ]:
os.chdir('..')

In [ ]:
xl_importer = bw.ExcelImporter("output/bw_input_sheet.xlsx")

In [ ]:
xl_importer.apply_strategies()

In [ ]:
xl_importer.statistics()

In [ ]:
for obj in xl_importer.unlinked:
    print(obj)

In [ ]:
# Match FG_DB to itself
xl_importer.match_database(fields=['name', 'unit'])
xl_importer.statistics()

In [ ]:
# Match FG_DB to the biosphere
xl_importer.match_database(fields=["name", "categories"])
xl_importer.match_database("biosphere3", fields=["name", "categories"])
xl_importer.statistics()

In [ ]:
# Match FG_DB to ecoinvent
xl_importer.match_database(background_database_name, fields=["name", "unit", "location"])
#xl_importer.match_database("ecoinvent-3.10-cutoff")
xl_importer.statistics()

## Unlinked activities

In [ ]:
for obj in xl_importer.unlinked:
    display(obj)

## Search for unlinked processes in DB

In [ ]:
eidb = bw.Database(background_database_name)
searched_process = eidb.search('electricity')
searched_process

In [ ]:
eidb = bw.Database('biosphere3')
searched_process = eidb.search('gold')
searched_process

## Write spreadsheet and database

In [ ]:
# Check data (optional)
xl_importer.write_excel()

In [ ]:
xl_importer.write_database()

In [ ]:
bw.databases

# LCI

In [ ]:
fgdb = bw.Database(product_system_database_name)

In [ ]:
process1 = [act for act in fgdb if 'synthesis gas, photocatalysis' in act['name'] and 'RER' in act['location']][0]
prod_sys = [{process1.key:1}]
prod_sys

# IMPACT ASSESSMENT METHODS

In [ ]:
bw.methods

In [ ]:
methods = [[m for m in bw.methods if 'EF v3.1' in str(m) and 'climate change' in str(m)  and not 'biogenic' in str(m) and not 'fossil' in str(m) and not 'land' in str(m) and not 'no LT' in str(m)][0],
           [m for m in bw.methods if 'EF v3.1' in str(m) and  'land use' in str(m) and not 'climate' in str(m) and not 'no LT' in str(m) ][0],
           [m for m in bw.methods if 'EF v3.1' in str(m) and  'metals/minerals' in str(m) and not 'no LT' in str(m) and not 'V1.13' in str(m)][0]]
methods

# CALCULATION

In [ ]:
bw.calculation_setups['multiLCA'] = {'inv': prod_sys, 'ia': methods}
bw.calculation_setups['multiLCA']
myMultiLCA = bw.MultiLCA('multiLCA')
myMultiLCA.results

# LCIA

In [ ]:
all_scores = {}
product_lca = bw.LCA({process1:1}, methods[0])
product_lca.lci()
product_lca.lcia()
for category in methods:
    product_lca.switch_method(category)
    product_lca.lcia()
    all_scores[category] = {}
    all_scores[category]['score'] = product_lca.score
    all_scores[category]['unit'] = bw.Method(category).metadata['unit']
    print("The score is {:f} {} for impact category {}".format(product_lca.score, 
                                                 bw.Method(category).metadata['unit'],
                                                 bw.Method(category).name)
    )
   
df = pd.DataFrame.from_dict(all_scores).T
df

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
df_m2 = df[df['unit']=='kg CO2-Eq']
df_m2

In [ ]:
df_m2.plot(kind='barh')